Develop a program for depthwise and pointwise convolutions. The program should take an initial image, kernel (filter), and produce the convoluted image based on the external flag that indicates either the depthwise or pointwise convolution.

In [1]:
import numpy as np
from scipy.signal import correlate2d


def conv2d_single_channel(image, kernel):
    """Perform 2D convolution for a single channel."""
    return correlate2d(image, kernel, mode='valid')


def depthwise_convolution(image, kernels):
    """
    Perform depthwise convolution.
    image: numpy array of shape (H, W, C_in)
    kernels: numpy array of shape (C_in, kH, kW)
    returns: numpy array of shape (H - kH + 1, W - kW + 1, C_in)
    """
    H, W, C = image.shape
    kC, kH, kW = kernels.shape
    assert C == kC, "Number of kernels must equal number of channels."

    outputs = []
    for c in range(C):
        out = conv2d_single_channel(image[:, :, c], kernels[c])
        outputs.append(out)
    return np.stack(outputs, axis=-1)


def pointwise_convolution(image, kernels):
    """
    Perform pointwise convolution (1x1).
    image: numpy array of shape (H, W, C_in)
    kernels: numpy array of shape (C_out, C_in, 1, 1)
    returns: numpy array of shape (H, W, C_out)
    """
    H, W, C_in = image.shape
    C_out, Ck, _, _ = kernels.shape
    assert C_in == Ck, "Input channels must match kernel input channels."

    outputs = np.zeros((H, W, C_out))
    for i in range(H):
        for j in range(W):
            pixel = image[i, j, :]  # vector of size C_in
            for co in range(C_out):
                weight = kernels[co, :, 0, 0]
                outputs[i, j, co] = np.sum(pixel * weight)
    return outputs


def depthwise_pointwise_convolution(image, kernels, mode='depthwise'):
    """
    Main function to perform convolution based on flag.
    mode: 'depthwise' or 'pointwise'
    """
    if mode == 'depthwise':
        return depthwise_convolution(image, kernels)
    elif mode == 'pointwise':
        return pointwise_convolution(image, kernels)
    else:
        raise ValueError("Invalid mode. Use 'depthwise' or 'pointwise'.")


# ------------------- Example Usage -------------------

if __name__ == "__main__":
    # Example image: 4x4 with 3 channels
    img = np.random.rand(4, 4, 3)

    # Depthwise kernel: one 3x3 filter per channel
    depthwise_kernels = np.random.randn(3, 3, 3)  # (C_in, kH, kW)

    # Pointwise kernel: say we want 2 output channels
    pointwise_kernels = np.random.randn(2, 3, 1, 1)  # (C_out, C_in, 1, 1)

    # Perform depthwise
    depth_out = depthwise_pointwise_convolution(img, depthwise_kernels, mode='depthwise')
    print("Depthwise output shape:", depth_out.shape)

    # Perform pointwise
    point_out = depthwise_pointwise_convolution(img, pointwise_kernels, mode='pointwise')
    print("Pointwise output shape:", point_out.shape)


Depthwise output shape: (2, 2, 3)
Pointwise output shape: (4, 4, 2)
